In [ ]:
import numpy
import sys
import pymzml
import scipy
import os
import math
import tensorly
import peakutils
import matplotlib
from scipy import signal
from matplotlib import pyplot
import glob
raw_array = []
raw_time = []
raw_mass = []
final_denoised_corrected_HR_array = []
final_corrected_masses = []
TIC = []
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
labels = []


#reading files
for filename in sorted(os.listdir(path = "/Users/polina_turova/Documents/test"), key=numericalSort):
    IS_time_array = []
    IS_mass_array = []
    print (filename)
    labels.append(filename[:-5])
    run = pymzml.run.Reader('/Users/polina_turova/Documents/test/' + filename)
    intensities = []
    mass = []
    time = numpy.empty([0])
    tic = numpy.empty([0])
    for spectrum in run:
        time = numpy.append(time, round(spectrum.scan_time_in_minutes(),3))
        mass.append(spectrum.mz)
        peaks = spectrum.peaks("raw")[:,1]
        intensities.append (peaks)
        tic = numpy.append (tic, spectrum.TIC )
    
    #creation of arrays with raw data
    raw_array.append(intensities)    
    raw_time.append(time)
    raw_mass.append(mass)
    TIC.append(tic)
    
    #redusing the number of peaks to only those which intensity is higher than 5%
    valuable_mass = []
    intensity_valuable_mass = []
    denoised_array = [None] * len(intensities)
    denoised_mass = [None] * len(mass)
    for i in range (len(mass)):
        percents = intensities[i] / max(intensities[i])
        for j in range (len(mass[i])):
            if percents[j] > 0.05:
                valuable_mass.append(mass[i][j])
                intensity_valuable_mass.append(intensities[i][j])
        denoised_mass[i] = valuable_mass
        denoised_array[i] = intensity_valuable_mass
        valuable_mass = []
        intensity_valuable_mass = []
        
    #mass correction due to IS mass    
    max_IS = 0
    for i in range(len(time)):
        for j in range (len(denoised_mass[i])):
            if (denoised_mass[i][j] > 395.12 and denoised_mass[i][j] < 395.13):
                if denoised_array[i][j] > max_IS:
                    max_IS = denoised_array[i][j]
                    time_IS = time[i]
                    mass_IS = denoised_mass[i][j]
    delta = 395.13180 - mass_IS
    denoised_mass = numpy.array(denoised_mass,dtype=object)
    denoised_array = numpy.array(denoised_array,dtype=object)
    for i in range (len(denoised_mass)):
        denoised_mass[i] = numpy.array(denoised_mass[i])
        denoised_array[i] = numpy.array(denoised_array[i])
    corrected_mass = denoised_mass + delta
    
    #apppending final arrays to whole dataset
    final_denoised_corrected_HR_array.append(denoised_array)
    final_corrected_masses.append(corrected_mass)